In [ ]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import h5py

import os
import numpy as np
import random
import datetime
import io
import json
import keras
import string
import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation, Conv2DTranspose, Add
from keras.layers import LeakyReLU, UpSampling2D, ZeroPadding2D, GaussianNoise
from keras.optimizers import Adam

y_size=751
x_size=128
num_channels=3

def get_model(y_size, x_size, num_channels):

    input_graph = Input(shape=(y_size, x_size, num_channels), name='input_image')

    graph = GaussianNoise(0.1)(input_graph)
    
    #block_1
    graph = Conv2D(8, 3, padding='same', name='block1_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(8, 3, padding='same', name='block1_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = MaxPooling2D(pool_size=(4, 2), padding='valid', name='block1_pool')(graph)
    
    #block_2
    graph = Conv2D(16, 3, padding='same', name='block2_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(16, 3, padding='same', name='block2_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = MaxPooling2D(pool_size=(3, 2), padding='valid', name='block2_pool')(graph)

    #block_3
    graph = Conv2D(32, 3, padding='same', name='block3_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(32, 3, padding='same', name='block3_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = MaxPooling2D(pool_size=(3, 2), padding='valid', name='block3_pool')(graph)

    #block_4
    graph = Conv2D(64, 3, padding='same', name='block4_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(64, 3, padding='same', name='block4_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = MaxPooling2D(pool_size=(3, 2), padding='valid', name='block4_pool')(graph)
        
    # Middle block
    graph = Conv2D(128, 3, padding='same', name='middle_block_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = ZeroPadding2D(((1, 1), (0, 0)))(graph)

    #deconvolution block 4
    graph = UpSampling2D(size=(2, 2))(graph)
    graph = Conv2D(64, 3, padding='same', name='deconv_block4_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(64, 3, padding='same', name='deconv_block4_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
        
    #deconvolution block 3
    graph = UpSampling2D(size=(2, 2))(graph)
    graph = Conv2D(32, 3, padding='same', name='deconv_block3_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(32, 3, padding='same', name='deconv_block3_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)

    #deconvolution block 2
    graph = UpSampling2D(size=(2, 2))(graph)
    graph = Conv2D(16, 3, padding='same', name='deconv_block2_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(16, 3, padding='same', name='deconv_block2_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)

    #deconvolution block 1
    graph = UpSampling2D(size=(2, 2))(graph)
    graph = Conv2D(8, 3, padding='same', name='deconv_block1_conv1')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(8, 3, padding='same', name='deconv_block1_conv2')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
    graph = Conv2D(1, 3, padding='same', name='deconv_block1_conv3')(graph)
    graph = LeakyReLU()(graph)
    graph = BatchNormalization()(graph)
   
    output_graph = Activation('linear')(graph)

    return Model(input_graph, output_graph)